In [1]:
import pandas as pd

In [2]:
# Load the CSV file into a DataFrame
prices = pd.read_csv('agile-half-hour-actual-rates-01-10-2023_30-09-2024.csv')

# Drop the 'Period to' column
prices.drop(columns=["Period to"], inplace=True)

# Convert the 'Period from' column to datetime format
prices['Period from'] = pd.to_datetime(prices['Period from'], format='%d/%m/%Y %H:%M')

# Check for missing dates
missing_dates = pd.date_range(start='2023-10-01 00:00', end='2024-09-30 23:30', freq='30min').difference(prices['Period from'])
print("Missing dates:", missing_dates)

Missing dates: DatetimeIndex(['2024-03-31 01:00:00', '2024-03-31 01:30:00'], dtype='datetime64[ns]', freq='30min')


In [3]:
# Set the 'Period from' column as the index
prices.set_index('Period from', inplace=True)

In [4]:
avg_import_price = (prices.loc[missing_dates[0] - pd.Timedelta(minutes=30), 'Agile Import price (p/kWh)'] + prices.loc[missing_dates[-1] + pd.Timedelta(minutes=30), 'Agile Import price (p/kWh)']) / 2
avg_export_price = (prices.loc[missing_dates[0] - pd.Timedelta(minutes=30), 'Agile Export price (p/kWh)'] + prices.loc[missing_dates[-1] + pd.Timedelta(minutes=30), 'Agile Export price (p/kWh)']) / 2

In [5]:
# Create a new row with the average values
new_rows = pd.DataFrame({
    'Agile Import price (p/kWh)': [avg_import_price, avg_import_price],
    'Agile Export price (p/kWh)': [avg_export_price, avg_export_price],
}, index=[missing_dates[0], missing_dates[-1]])

# Append the new rows to the DataFrame
prices = pd.concat([prices, new_rows])

# Sort the DataFrame by the index (Date)
prices.sort_index(inplace=True)

In [6]:
prices.loc[missing_dates[0] - pd.Timedelta(minutes=60):missing_dates[-1] + pd.Timedelta(minutes=60)]

,Agile Import price (p/kWh),Agile Export price (p/kWh)
2024-03-31 00:00:00,13.89,6.86
2024-03-31 00:30:00,16.10,7.80
2024-03-31 01:00:00,17.07,8.22
2024-03-31 01:30:00,17.07,8.22
2024-03-31 02:00:00,18.04,8.64
2024-03-31 02:30:00,16.32,7.90


In [7]:
# Reset the index
prices.reset_index(inplace=True)
prices.rename(columns={'index': 'Date'}, inplace=True)

In [8]:
simulated_data = pd.read_csv("./simulation_results_finished.csv")

In [9]:
simulated_data.drop(columns=["Agile Import price (p/kWh)", "Agile Export price (p/kWh)"], inplace=True)

In [10]:
simulated_data.set_index('Date', inplace=True)

In [11]:
# Assuming simulated_data is already loaded
# Convert the index to datetime
simulated_data.index = pd.to_datetime(simulated_data.index)

# Replace the year 2023 with 2024
simulated_data.index = simulated_data.index.map(lambda x: x.replace(year=2024))

# Sort the DataFrame by the index (Date)
simulated_data.sort_index(inplace=True)

In [12]:
date2_28 = simulated_data.loc["2024-02-28 00:00:00":"2024-02-28 23:30:00"]
date2_28.reset_index(inplace=True)
date3_1 = simulated_data.loc["2024-03-01 00:00:00":"2024-03-01 23:30:00"]
date3_1.reset_index(inplace=True)

In [13]:
# Assuming simulated_data is already loaded
# Calculate the average values for each column between 2024-02-28 and 2024-03-01
avg_values = (date2_28.iloc[:, 1:] + date3_1.iloc[:, 1:]) / 2

# Generate a range of datetime objects for 2024-02-29 with half-hourly intervals
new_dates = pd.date_range(start='2024-02-29 00:00:00', end='2024-02-29 23:30:00', freq='30T')

# Create a new DataFrame with the new dates and the average values
# new_data = pd.DataFrame(avg_values, index=new_dates)
new_data = avg_values.set_index(new_dates)

# Append the new DataFrame to the existing simulated_data DataFrame
simulated_data = pd.concat([simulated_data, new_data])

# Sort the DataFrame by the index (Date)
simulated_data.sort_index(inplace=True)

/var/folders/rr/ntbzcq0130b7swcl62nshrfc0000gn/T/ipykernel_70592/2642770.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  new_dates = pd.date_range(start='2024-02-29 00:00:00', end='2024-02-29 23:30:00', freq='30T')


In [14]:
date_p1 = simulated_data.loc["2024-01-01 00:00:00":"2024-09-30 23:30:00"]
date_p2 = simulated_data.loc["2024-10-01 00:00:00":"2024-12-31 23:30:00"]
reordered_simulated_data = pd.concat([date_p2, date_p1])

In [15]:
reordered_simulated_data.reset_index(inplace=True)
reordered_simulated_data.rename(columns={'index': 'DateTime'}, inplace=True)

In [16]:
# Concatenate DataFrames horizontally
results = pd.concat([reordered_simulated_data, prices], axis=1)

In [17]:
results.drop(columns=["DateTime"], inplace=True)

In [18]:
# Set the 'Date' column as the index
results.set_index('Date', inplace=True)

In [19]:
results.head()

,Dry-bulb temperature (°C),Total system energy (kBtu/h),System natural gas (kBtu/h),System electricity (kBtu/h),Total lights energy (kBtu/h),Lights electricity (kBtu/h),Total equip energy (kBtu/h),Equip electricity (kBtu/h),Total CE ex equip (kgCO2/h),Total system CE (kgCO2/h),...,Total electricity (kBtu/h),Total grid displaced elec. PV (kBtu/h),Total energy (kBtu/h),Transformer Losses (kBtu/h),DHW demand (kW),SH demand (kW),Total heat demand (kW),Ele demand (kW),Agile Import price (p/kWh),Agile Export price (p/kWh)
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-01 00:00:00,10.4,4.824,4.688,0.135,0.0,0.0,2.912,2.912,0.3,0.3,...,3.048,0.0,7.736,0.0,0.176006,0.662434,0.838439,0.228635,18.48,8.83
2023-10-01 00:30:00,10.4,4.824,4.688,0.135,0.0,0.0,2.912,2.912,0.3,0.3,...,3.048,0.0,7.736,0.0,0.176006,0.662434,0.838439,0.228635,17.86,8.56
2023-10-01 01:00:00,11.5,4.824,4.688,0.135,0.0,0.0,2.912,2.912,0.3,0.3,...,3.048,0.0,7.736,0.0,0.176006,0.687814,0.863820,0.228635,16.32,7.90
2023-10-01 01:30:00,11.5,4.824,4.688,0.135,0.0,0.0,2.912,2.912,0.3,0.3,...,3.048,0.0,7.736,0.0,0.176006,0.687814,0.863820,0.228635,15.17,7.41
2023-10-01 02:00:00,11.5,4.824,4.688,0.135,0.0,0.0,2.912,2.912,0.3,0.3,...,3.048,0.0,7.736,0.0,0.176006,0.704735,0.880740,0.228635,11.03,5.62


In [ ]:
# Export the DataFrame to a CSV file
# results.to_csv('simulation_results_2024.csv', index=True)